In [ ]:
import numpy as np
import pandas as pd
import easyocr
from collections import Counter
from PIL import Image, ImageDraw
import cv2

In [ ]:
df = pd.read_csv('./Data/dataset/european_images.csv', index_col=0)
df['img_path'] = [f"./Data/dataset/{index}.png" for index in df.index]
df['text'] =  pd.Series(dtype='object')
df['confidence'] =  pd.Series(dtype='object')
df['bbox'] =  pd.Series(dtype='object')
df['lang'] = pd.Series(None)
df['best_confidence'] = pd.Series(0.0)

Need to determine the best OCR model for this.

Candidates:
    easyocr (built in, uses pytorch)

# OCR Text Recognition
1. It's important to note that we're looking at a plethora of European languages, which will employ various modifications to the latin script, cyrillic, etc. 
2. As such, multiple OCR readers can be employed. We can run each reader on an image, extract the confidence and various pieces of text above some threshold, and then input the collected data to our classifier.
3. We could also use a reader with multiple languages in the list. This will remove the ability to handle different languages differently, but that may be better to avoid data leakage?
4. It may be necessary to prune the 'Google' labels from the images.


## OCR Text Recognition: Language definition

https://www.jaided.ai/easyocr/

In [ ]:
class ocr_reader():
    def __init__(self, lang):
        self.lang = lang
        self.reader = easyocr.Reader([lang])
    
    def __call__(self, df):
        min_length = 3
        self.df_results = df.copy()
        
        for index, row in self.df_results.iterrows():
            
            results = self.OCR_extraction(row['img_path'], self.reader)
            
            bbox_ = []
            text = []
            confidence = []
            for result in results:
                if len(result[1]) >= min_length:
                    bbox_.append(result[0])
                    text.append(result[1])
                    confidence.append(result[2])
                
            self.df_results.at[index, "bbox"] = bbox_
            self.df_results.at[index, "text"] = text
            self.df_results.at[index, "confidence"] = confidence
        
        return self.df_results
    def OCR_extraction(self, path, reader):
        results =  reader.readtext(path)
        return results
    

In [ ]:
languages = {
    "be": "be",
    "bg": "bg",
    "cs": "cs",
    "cy": "cy",
    "da": "da",
    "de": "de",
    "en": "en",
    "es": "es",
    "et": "et",
    "fr": "fr",
    "ga": "ga",
    "hr": "hr",
    "hu": "hu",
    "is": "is",
    "it": "it",
    "la": "la",
    "lt": "lt",
    "lv": "lv",
    "mt": "mt",
    "nl": "nl",
    "no": "no",
    "pl": "pl",
    "pt": "pt",
    "ro": "ro",
    "ru": "ru",
    "rs_latin": "rs",
    "rs_cyrillic": "rc",
    "sk": "sk",
    "sl": "sl",
    "sq": "sq",
    "sv": "sv",
    "tr": "tr",
    "uk": "uk"
}

Readers = {}

for key, lang in languages.items():
    Readers[lang] = ocr_reader(key)

Either only store text above a certain threshold for all, or only store text above a certain threshold for each language.
On CPU, roughly 3s to initialize the OCR reader.

### Strip Google watermarks from detected text

Generating text for each image for each reader, then using manual pruning to determine which labels to drop for each language.
These can then be associated with the correct language.

I use the english reader, identify matching watermark text, determine the bounding box, and mask the image for that watermark.



In [ ]:
def OCR_extraction(path, reader):
    results =  reader.readtext(path)
    return results

    

In [ ]:

import os
os.makedirs("./Data/masked_for_ocr/", exist_ok=True)

reader_en = Readers["en"].reader

watermarks = ["cgoogle", "cgccgle", "cgcogle", "cgocgle", "google", "gccgle", "gcogle", "gocgle", "gocnle", "gcogie", "oogle"]

### For each image, generate a mask (initially blank). Use this to mask results for the other readers.
for index, row in df.iterrows():
    
    ### Generating the mask with the english reader, filtering out the sections of the image containing the "Google" watermark.
    image = cv2.imread(row['img_path'])
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255
    results = OCR_extraction(row['img_path'], reader_en)
    
    bbox_ = []
    text = []
    confidence = []
    for result in results:
        bbox_.append(result[0])
        text.append(result[1])
        confidence.append(result[2])
    
    for idx, t in enumerate(text):
        ### Filtereing text matching the filter, or text with less than min_length characters (meaningless)
        min_length = 3
        if any(watermark.upper() in t.upper() for watermark in watermarks) or len(t) <= min_length:
            corners = bbox_[idx]
            x0, y0 = np.array(corners[0], dtype=np.int32) ### Top left corner
            x1, y1 = np.array(corners[2], dtype=np.int32) ### Bottom right corner
            cv2.rectangle(mask, (x0, y0), (x1, y1), 0, -1)
        else:
            print(f"Text {t.upper()} passed the filter for img index {index} and confidence {confidence[idx]}")
    
    ### We now have a masked image instead of the original image. Other lang readers can now use this image.
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    ### Store the images in a new directory (masked_for_ocr/) so that I only have to do this once
    cv2.imwrite(f"./Data/masked_for_ocr/{index}.png", masked_image)    
    

In [ ]:
masked_df = pd.read_csv('./Data/dataset/european_images.csv', index_col=0)
masked_df['img_path'] = [f"./Data/masked_for_ocr/{index}.png" for index in df.index]
masked_df['text'] =  pd.Series(dtype='object')
masked_df['confidence'] =  pd.Series(dtype='object')
masked_df['bbox'] =  pd.Series(dtype='object')

##### Now, with the text extracted from the masked images, we can set confidence thresholds, and only keep results above specific thresholds.

##### Identify the language with the highest confidence, and use this as our guessed label
##### Additionally, we can extract text and pass forward

In [ ]:
#
Full_Results = {}
for lang, reader in Readers.items():
    Full_Results[lang] = reader(masked_df)

#### Pass extracted text through LLM to predict country or language
Here I am following coding worst practices

In [ ]:
THRESHOLD = 0.1
### For each row, iterate through the language DF and identify the highest confidence text
### If no text was found, "best_lang" = None and thus df['lang'] = None
for idx, _ in df.iterrows():
    best_confidence = 0.0
    best_lang = None
    best_text = ""
    for lang, lang_df in Full_Results.items():
        if idx in lang_df.index:
            row = lang_df.loc[idx]
            if row['text'] is not None and row['text'] != []:
                ### Handle single and full
                #print(f"Row confidence: {row['confidence']}")
                cidx = np.argmax(row['confidence'])
                if row['confidence'][cidx] > best_confidence:
                    best_confidence = row['confidence'][cidx]
                    best_lang = lang
                    best_text = row['text'][cidx]
    if best_confidence > THRESHOLD:
    
        df.at[idx, 'best_confidence'] = best_confidence
        df.at[idx, 'lang'] = best_lang
        print(f"For image {idx}, lang found is {best_lang} with text {best_text}")

print
df.to_csv('./Outputs/OCR_results.csv', index=True)

### Pass this through feedforward, or to LLM? Could just tokenize it and pass it forward

Saving the lang data for OH

In [ ]:
import json
with open('./Outputs/languages.json', 'w') as json_file:
    json.dump(languages, json_file, indent = 4)